# Azure Cosmos DB - Flight Data Queries

This notebook demonstrates query patterns for flight data in Azure Cosmos DB, including filtering, aggregations, and analytics.

**Prerequisites:**
- Run the `cosmosdb-insert.ipynb` notebook first to seed the data
- Ensure you have a valid .env file with Cosmos DB connection details

## Setup

Install required packages and configure the Cosmos DB connection.

**Note:** If you've already run the insert notebook in this session, you may skip the setup steps.

**Step 1**: Install NuGet packages

**IMPORTANT**: If you get version conflict errors, restart the kernel first (click "Clear All Outputs" or restart the notebook kernel).

We'll use:
- `Microsoft.Azure.Cosmos` - Azure Cosmos DB SDK for .NET
- `DotNetEnv` - Load environment variables from .env file
- `System.Text.Json` - JSON serialization

In [ ]:
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: System.Text.Json, 8.0.0"
#r "nuget: Microsoft.Azure.Cosmos, 3.43.1"

**Step 2**: Load environment variables from .env file

The .env file in the root directory contains connection strings and configuration.

In [ ]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Directory.GetCurrentDirectory(), "..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from .env");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

var cosmosEndpoint = Environment.GetEnvironmentVariable("COSMOS_DB_ENDPOINT");
var databaseName = Environment.GetEnvironmentVariable("COSMOS_DB_DATABASE_NAME");
var flightsContainerName = Environment.GetEnvironmentVariable("COSMOS_DB_FLIGHTS_CONTAINER") ?? "Flights";

Console.WriteLine($"Cosmos DB Endpoint: {cosmosEndpoint}");
Console.WriteLine($"Database: {databaseName}");
Console.WriteLine($"Container: {flightsContainerName}");

**Step 3**: Connect to Cosmos DB

Establish connection using the connection string or endpoint/key pair.

In [ ]:
using Microsoft.Azure.Cosmos;
using System.Diagnostics;

CosmosClient client = null;
var cosmosConnectionString = Environment.GetEnvironmentVariable("COSMOS_DB_CONNECTION_STRING");

client = new CosmosClient(cosmosConnectionString);

var flightsContainer = client.GetContainer(databaseName, flightsContainerName);
Console.WriteLine($"Container reference obtained: {flightsContainerName}");

---

## Querying Flight Data

Let's query the flight data using various filters and aggregations.

**Query 1**: Find all flights from Melbourne to Auckland

In [ ]:
var originCode = "MEL";
var destinationCode = "AKL";

var routeQuery = @"
SELECT 
    c.flightNumber,
    c.airline.name as airlineName,
    c.route.origin.city as origin,
    c.route.destination.city as destination,
    c.pricing.amount as price,
    c.pricing.currency as currency,
    c.stops
FROM c 
WHERE c.type = 'flight'
AND c.route.origin.code = @origin
AND c.route.destination.code = @destination
ORDER BY c.pricing.amount
";

var queryDef = new QueryDefinition(routeQuery)
    .WithParameter("@origin", originCode)
    .WithParameter("@destination", destinationCode);

var iterator = flightsContainer.GetItemQueryIterator<dynamic>(queryDef);

Console.WriteLine($"Flights from {originCode} to {destinationCode}:\n");
Console.WriteLine(new string('=', 80));

var count = 0;
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    foreach (var flight in response)
    {
        count++;
        var stopsText = flight.stops == 0 ? "Direct" : $"{flight.stops} stop(s)";
        Console.WriteLine($"\n{count}. {flight.airlineName} {flight.flightNumber}");
        Console.WriteLine($"   {flight.origin} → {flight.destination}");
        Console.WriteLine($"   ${flight.price:F0} {flight.currency} | {stopsText}");
    }
}

Console.WriteLine($"\n{new string('=', 80)}");
Console.WriteLine($"Found {count} flight(s)");

**Query 2**: Find budget flights (under $1,500) from Melbourne

In [ ]:
var maxBudget = 1500;
var originCode = "MEL";

var budgetQuery = @"
SELECT 
    c.flightNumber,
    c.airline.name as airlineName,
    c.route.destination.city as destination,
    c.pricing.amount as price,
    c.stops,
    ARRAY_LENGTH(c.amenities) as amenityCount
FROM c 
WHERE c.type = 'flight'
AND c.route.origin.code = @origin
AND c.pricing.amount <= @maxBudget
ORDER BY c.pricing.amount
";

var budgetQueryDef = new QueryDefinition(budgetQuery)
    .WithParameter("@origin", originCode)
    .WithParameter("@maxBudget", maxBudget);

var budgetIterator = flightsContainer.GetItemQueryIterator<dynamic>(budgetQueryDef);

Console.WriteLine($"Budget flights from {originCode} (under ${maxBudget}):\n");
Console.WriteLine(new string('=', 80));

var count = 0;
while (budgetIterator.HasMoreResults)
{
    var response = await budgetIterator.ReadNextAsync();
    foreach (var flight in response)
    {
        count++;
        var stopsText = flight.stops == 0 ? "Direct" : $"{flight.stops} stop(s)";
        Console.WriteLine($"\n{count}. {flight.airlineName} {flight.flightNumber} → {flight.destination}");
        Console.WriteLine($"   ${flight.price:F0} | {stopsText} | {flight.amenityCount} amenities");
    }
}

Console.WriteLine($"\n{new string('=', 80)}");
Console.WriteLine($"Found {count} budget flight(s)");